# IVP Solution for a single Mode:


Solution is given by

\begin{equation}
\epsilon\frac{\partial\theta}{\partial t} + \cos{(y)}\frac{\partial\theta}{\partial x} = \frac{1}{Pe}\frac{\partial^2\theta}{\partial x^2} + \epsilon\frac{\partial^2\theta}{\partial y^2}
\end{equation}


with the initial condition

\begin{equation}
\theta(x, y, 0) = \cos{(k_{0}x)}
\end{equation}

\begin{equation}
\theta(x, y, t) = \theta_{0}Re\left\{\sum_{n=0}^{\infty}2A_{0}^{(2n)}ce_{2n}\exp{\left[ ik_{0}x - \left(\frac{a_{2n}}{4} + \frac{k_{0}^2}{\epsilon Pe} \right)t\right]}\right\}
\end{equation}





In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from mathieu_functions import A_coefficients
from mathieu_functions import mathieu_functions as mfs

In [ ]:
import holoviews as hv
hv.extension('bokeh')

# Important parameters to define


In [ ]:
# =================================
L = np.pi # Half of channel width (y-direction)
alpha = 10  # length of channel periodic in x. I have used alpha=10 before, but for the gaussian initial condition a value of 2 is better.
Nx = 500  # length of x-array


# =============================
#  Change only parameters
# =============================
eps = 0.05
k = .1 # lowest possible is 1 / alpha
Pe = 1 / eps

x = np.linspace(-alpha * L, alpha * L, Nx)
y = np.linspace(0, L, Nx//5)
X, Y = np.meshgrid(x, y)

qf = 2 * k / eps
Nq = 100
Q = np.linspace(0, qf, Nq)
Q = Q * (1j)

t = np.linspace(0, 2.5, 200)

In [ ]:
print('Value of parameter q is:', (qf * (1j)))

In [ ]:
print('Mode of initial condition (k_0):', k)

In [ ]:
N = 25  # matrix size
A_vals = A_coefficients(Q, N, 'even', 'one')
vals = mfs.ce_even(Q, y, N, As=A_vals)

In [ ]:
OM = []  # Initialize list containing Mathieu Eigenvalues
AA = []  # Initialize list containing zeroth-Fourier Coefficients 
CE = []  # Initialize list containing Mathieu Eigenfunctions
for n in range(N // 2):
    OM.append(0.25*np.copy(A_vals['a' + str(2 * n)]))
    AA.append(np.copy(A_vals['A' + str(2 * n)][:, 0]))
    ce = np.repeat(vals['ce'+str(2 * n)][:, :, np.newaxis], Nx, axis=2)
    CE.append(ce)

In [ ]:
CE2n0 = 0
for n in range(N//2):
    CE2n0 = CE2n0 + 2 * A_vals['A'+str(2 * n)][-1, 0] * CE[n][-1, :, :]
T0 = np.real(np.exp((k * X) * (1j)) * CE2n0)
T0 = T0[np.newaxis, :]

In [ ]:
## Add temporal contribution
for i in range(1, len(t)):
    CE2n = 0
    for n in range(N//2):
        CE2n = CE2n + 2 * A_vals['A'+str(2 * n)][-1, 0] * CE[n][-1, :, :] * np.exp(-(A_vals['a'+str(2 * n)][-1]/4) * t[i])
    t0 = np.real(np.exp((k * X) * (1j)) * CE2n)
    t0 = t0[np.newaxis, :]
    T0 = np.append(T0, t0, axis=0)

## Make Animation

In [ ]:
Temp = xr.DataArray(T0, coords=[t, y, x], dims=["time", 'y', 'x'])
ds = xr.Dataset({'Theta': Temp})

In [ ]:
%%output holomap='scrubber'
%%opts Image style(cmap='PRGn') plot[colorbar=True]
%%opts Image [width=600, height=450]
hv_ds = hv.Dataset(ds.Theta.isel(time=slice(0, -1, 4), x=slice(0,-1, 2)))
hv_ds.to(hv.Image, ['x', 'y'])